In [1]:
import pandas as pd
data_dir = '/nfs/datasets/cxj/retrievability-bias/data/queries'
sampled_df = pd.read_csv(f'{data_dir}/random_sampled_docs_100k_from_msmarco.csv', index_col=0).reset_index()


In [ ]:
from pyterrier_doc2query import Doc2Query, QueryScorer, QueryFilter
from pyterrier_dr import ElectraScorer
# doc2query = Doc2Query(append=False, num_samples=5)
# scorer = ElectraScorer()
# pipeline = doc2query >> QueryScorer(scorer) >> QueryFilter(t=3.21484375)
# pipeline(sampled_df[:2])

In [ ]:
df = sampled_df[:2]

In [ ]:
df

In [9]:
from transformers import pipeline
generator = pipeline("text2text-generation", model="macavaney/doc2query-t5-base-msmarco")

Device set to use cuda:0


In [12]:
def generate_queries(text, num_queries=3):
    prompt = f"generate queries: {text}"
    return [q['generated_text'] for q in generator(prompt, num_return_sequences=num_queries, max_length=64, do_sample=True,
          top_k=50,)]

In [26]:
df['generated_queries'] = df['text'].apply(lambda x: generate_queries(x, num_queries=3))


In [28]:
from sentence_transformers import CrossEncoder

scorer = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

def score_queries(doc, queries):
    pairs = [(query, doc) for query in queries]
    scores = scorer.predict(pairs)
    return scores

In [29]:
df['query_scores'] = df.apply(lambda row: score_queries(row['text'], row['generated_queries']), axis=1)

In [30]:
def select_top_query(queries, scores):
    best_idx = scores.index(max(scores))
    return queries[best_idx]

In [31]:
df['selected_query'] = df.apply(lambda row: select_top_query(row['generated_queries'], list(row['query_scores'])), axis=1)

In [32]:
df

,doc_id,text,generated_queries,query_scores,selected_query
0,d1,This is the first document.,"[how to generate queries, excel generate a que...","[1.2218982e-05, 5.7999936e-05, 0.00028478578]",which query generates a document
1,d2,Second document goes here.,[in which document to use instead of generate ...,"[0.0013623098, 1.3263578e-05, 1.6412321e-05]",in which document to use instead of generate q...
